In [11]:
import torch
from model import load_model, generate, detect
from inputs import sports_input, technology_input, animals_input
from baseline import BaselineAttack


In [8]:
with open('./watermarked_outputs/sports_watermarked.txt', 'r') as file:
    content = file.read()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = {
    'demo_public': False, 
    # 'model_name_or_path': 'facebook/opt-125m', 
    'model_name_or_path': 'facebook/opt-1.3b', 
    # 'model_name_or_path': 'facebook/opt-2.7b', 
    # 'model_name_or_path': 'facebook/opt-6.7b',
    # 'model_name_or_path': 'facebook/opt-13b',
    # 'load_fp16' : True,
    'load_fp16' : False,
    'prompt_max_length': None, 
    'max_new_tokens': 200, 
    'generation_seed': 123, 
    'use_sampling': True, 
    'n_beams': 1, 
    'sampling_temp': 0.7, 
    'use_gpu': True, 
    'seeding_scheme': 'simple_1', 
    'gamma': 0.7, 
    'delta': 2.0, 
    'normalizers': '', 
    'ignore_repeated_bigrams': False, 
    'detection_z_threshold': 1.0, 
    'select_green_tokens': True,
    'skip_model_load': False,
    'seed_separately': True,
    'is_topic': True,
    'topic_token_mapping': {
        "sports": list(range(20000)),
        "animals": list(range(20000, 40000)),
        "technology": list(range(40000, 60000)),
        # Add more topics and corresponding tokens as needed
    },
    'detected_topic': "",
}

In [9]:
input_text = sports_input()
model, tokenizer = load_model(args)

input_prompt = input_text + content

with_watermark_detection_result = detect(input_prompt, content, 
                                        args, 
                                        device=device, 
                                        tokenizer=tokenizer)

print(with_watermark_detection_result)

[['Tokens Counted (T)', '200'], ['# Tokens in Greenlist', '154'], ['Fraction of T in Greenlist', '77.0%'], ['z-score', '2.16'], ['p value', '0.0154'], ['z-score Threshold', '1.0'], ['Prediction', 'Watermarked'], ['Confidence', '98.462%'], []]


In [12]:
baseline = BaselineAttack()
text = baseline.inference_modify_text(content, 3, 'insert')

input_prompt = input_text + text
with_watermark_detection_result = detect(input_prompt, text, 
                                        args, 
                                        device=device, 
                                        tokenizer=tokenizer)

print(with_watermark_detection_result)

[['Tokens Counted (T)', '204'], ['# Tokens in Greenlist', '158'], ['Fraction of T in Greenlist', '77.5%'], ['z-score', '2.32'], ['p value', '0.0101'], ['z-score Threshold', '1.0'], ['Prediction', 'Watermarked'], ['Confidence', '98.989%'], []]
